In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from collections import defaultdict

from pymongo import MongoClient
import pickle

In [3]:
def get_document(args):
    driver, title = args
    document = defaultdict(str)
    try:
        time.sleep(2)

        input_xpath = "//input[@id='masthead-search-term']"
        input = driver.find_element_by_xpath(input_xpath)
        input.clear()
        time.sleep(1)
        input.send_keys(title)
        input.send_keys(Keys.ENTER)
        time.sleep(2)
    except:
        print('!error not enough time %s' % title)
        return {}, title

    try:
        time.sleep(2)
        video_xpath = "//div[@class='yt-lockup-content' and div[@class='yt-lockup-byline']//a[@href='/user/TEDxTalks']]"
        video = driver.find_elements_by_xpath(video_xpath)
        ActionChains(driver).click(video[0]).perform()

        time.sleep(2)
    except:
        print('!!error connecting to video %s' % title)
        return {}, title

    try:
        # Viewership
        view_xpath = "//div[@class='watch-view-count']"
        views = driver.find_elements_by_xpath(view_xpath)
        document['viewership'] = views[0].text.split(' views')[0]

        # title
        title_xpath = "//span[@class='watch-title']"
        document['title'] = driver.find_elements_by_xpath(title_xpath)[0].text

        # Upload Date
        upload_day_xpath = "//strong[@class='watch-time-text']"
        document['upload_day'] = driver.find_elements_by_xpath(upload_day_xpath)[0].text.split('on ')[1]

        # UpVote, DownVote
        upvote_xpath = "//button[@title='I like this']//span[@class='yt-uix-button-content']"
        document['upvote'] = driver.find_elements_by_xpath(upvote_xpath)[0].text

        downvote_xpath = "//button[@title='I dislike this']//span[@class='yt-uix-button-content']"
        document['downvote'] = driver.find_elements_by_xpath(downvote_xpath)[0].text

        time.sleep(1)
        # Locate Transcript
        button_xpath = "//button[@id='action-panel-overflow-button']"
        buttons = driver.find_elements_by_xpath(button_xpath)
        ActionChains(driver).click(buttons[0]).perform()
        time.sleep(1)

        transcript_xpath = "//button[@data-trigger-for='action-panel-transcript']"
        transcripts = driver.find_elements_by_xpath(transcript_xpath)
        ActionChains(driver).click(transcripts[0]).perform()
        time.sleep(1)

        # Get Transcript
        lang_button_xpath = "//button[@class='yt-uix-button yt-uix-button-default hidden']//span[@class='yt-uix-button-content transcript-lang']"
        lang_button = driver.find_elements_by_xpath(lang_button_xpath)

        if lang_button[0].text not in ['English', 'English (Automatic Captions)']:
            ActionChains(driver).click(lang_button[0]).perform()
            time.sleep(1)

            lang_xpath = "//span[@class='yt-uix-button-content transcript-lang' and contains(text(),'English')]"
            lang = driver.find_elements_by_xpath(lang_xpath)

            if len(lang):
                ActionChains(driver).click(lang[0]).perform()
                time.sleep(1)
            else:
                print('language is not available in English')
                return {}, title

        text_xpath = "//div[@class='caption-line-text']"
        texts = driver.find_elements_by_xpath(text_xpath)

        lines = []
        [lines.append(text.text) for text in texts];
        doc = ' '.join(lines)
        document['transcript'] = doc

        print(document['title'])

    except:
        print('!!!Error processing %s ' % title)
        return {}, title
    return document, ''

In [4]:
chromedriver = "/Users/PandaGongfu/Downloads/chromedriver_2"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
youtube_url = "https://www.youtube.com"
driver = webdriver.Chrome(chromedriver)
driver.get(youtube_url)

In [9]:
pickle.dump(test_docs_list, open('tedx_scripts11.pickle', 'wb'))

In [ ]:
title_list = pickle.load(open('tedx_titles.pickle', 'rb'))
test_docs_list = []
error_load = []

count=0
test_links = title_list[15000:17500]

for link in test_links:
    print(count)
    count += 1
    document, error_link = get_document((driver, link))
    if len(error_link):
        error_load.append(error_link)
    else:
        test_docs_list.append(document)

0
